# ROB701@MBZUAI Fall 2025 - Lab 1009

**TA:** Tianyu Liu  
**Email:** [Tianyu.Liu@mbzuai.ac.ae](mailto:Tianyu.Liu@mbzuai.ac.ae)

**Topic:** 3R planar robot trajectory generation.

Note: Code sections pending implementation are labeled “[To Be Implemented]”.

<img src="https://raw.githubusercontent.com/RealGaule/ROB701_Lab/842ab4e9851bc4fcc03271e134283e67db776383/docs/Robotics.png" width="320">

In [ ]:
import os, sys, time
import numpy as np
import mujoco as mj
import mujoco.viewer as mjv
from types import SimpleNamespace
from scipy.optimize import fsolve
from scipy.interpolate import CubicSpline


print('MuJoCo version:', mj.__version__)


## 0.Three-Link Planar Robot
Suppose we have a 3-link planar robot with three revolute joints (3R). The following code defines a parameter class for this robot.

`Parameters` (class)

**Purpose:** Collects the physical properties of the 3R planar manipulator (all scalars, SI units).

**Fields & Physical Meaning**
- `m1`, `m2`, `m3` *(kg)* — mass of link 1/2/3.
- `Iz1`, `Iz2`, `Iz3` *(kg·m²)* — moment of inertia of each link about its joint axis (out of plane).
- `l1`, `l2`, `l3` *(m)* — geometric length of link 1/2/3.
- `g` *(m/s²)* — gravitational acceleration (default `9.81`).

**Notes**
- All angles are in **radians**; angular rates in **rad/s**; angular accelerations in **rad/s²**.
- Joint axes are assumed at inter-link connection points; the mechanism is planar with 3 revolute joints.
- The absolute reference of gravitational potential is conventional; only differences matter for energy conservation checks.

In [ ]:
import numpy as np

class Robot:
    class Body:
        def __init__(self, parent, name, pos, joint_axis, ipos, mass, inertia):
            self.parent = parent
            self.name = name
            self.pos = np.array(pos)
            self.joint_axis = np.array(joint_axis)
            self.ipos = np.array(ipos)
            self.mass = mass
            self.inertia = np.array(inertia)

    class Params:
        """Class to hold robot parameters with heterogeneous data types (scalar/array/matrix)."""
        def __setattr__(self, key, value):
            # Directly assign the value without conversion to preserve data types
            super().__setattr__(key, value)

        def __repr__(self):
            return str(self.__dict__)

    #functions in Robot class
    def __init__(self):
        self.body = {}
        self.params = Robot.Params()  # Use RobotParams for flexible parameter handling

    def add_body(self, body_id, parent, name, pos, joint_axis, ipos, mass, inertia):
        self.body[body_id] = Robot.Body(parent, name, pos, joint_axis, ipos, mass, inertia)


# Initialize the robot
robot = Robot()

# Add body
robot.add_body(
    1, parent='ground', name='link1', pos=[0, 0, 0], joint_axis=[0, 0, 1],
    mass=1, ipos=[0.5, 0, 0], inertia=[1, 1, 0.1, 0, 0, 0]
)

robot.add_body(
    2, parent='link1', name='link2', pos=[1, 0, 0], joint_axis=[0, 0, 1],
    mass=1, ipos=[0.5, 0, 0], inertia=[1, 1, 0.1, 0, 0, 0]
)

robot.add_body(
    3, parent='link2', name='link3', pos=[1, 0, 0], joint_axis=[0, 0, 1],
    mass=0.2, ipos=[0.125, 0, 0], inertia=[0.1, 0.1, 0.02, 0, 0, 0]
)

robot.params.end_eff_pos_local = np.array([0.25,0,0])
robot.params.w = 0.1;
robot.params.h = 0.1;
#print(robot.params.end_eff_pos_local)

#example usage
# Add robot-level heterogeneous parameters
# robot.params.array = np.array([0, 0, 0.125])  # Example: np.array
# robot.params.matrix = np.matrix([[1, 2], [3, 4]])  # Example: np.matrix
# robot.params.scalar = 42  # Example: scalar

# Access robot-level parameters
# print("Robot parameters:", robot.params)
# print("Array parameter:", robot.params.array)
# print("Matrix parameter:", robot.params.matrix)
# print("Scalar parameter:", robot.params.scalar)

# utilitity function

#### Module overview — utility function
Utility functions that transform among Quaternion, Euler Angles, and Rotation Matrix.

In [ ]:
_FLOAT_EPS = np.finfo(np.float64).eps
_EPS4 = _FLOAT_EPS * 4.0


def rotation(angle, axis):
    c, s = np.cos(angle), np.sin(angle)
    if axis == 0:  # Rotation around x-axis
        return np.array([
            [1, 0, 0],
            [0, c, -s],
            [0, s, c]
        ])
    elif axis == 1:  # Rotation around y-axis
        return np.array([
            [c, 0, s],
            [0, 1, 0],
            [-s, 0, c]
        ])
    elif axis == 2:  # Rotation around z-axis
        return np.array([
            [c, -s, 0],
            [s, c, 0],
            [0, 0, 1]
        ])
    else:
        raise ValueError("Axis must be 0 (x), 1 (y), or 2 (z).")


def quat2bryant(quat):
    """ Convert Quaternion to Euler Angles.  See rotation.py for notes """
    return mat2bryant(quat2mat(quat))

def bryant2quat(euler):
    """ Convert Euler Angles to Quaternions.  See rotation.py for notes """
    euler = np.asarray(euler, dtype=np.float64)
    assert euler.shape[-1] == 3, "Invalid shape euler {}".format(euler)

    ai, aj, ak = euler[..., 2] / 2, -euler[..., 1] / 2, euler[..., 0] / 2
    si, sj, sk = np.sin(ai), np.sin(aj), np.sin(ak)
    ci, cj, ck = np.cos(ai), np.cos(aj), np.cos(ak)
    cc, cs = ci * ck, ci * sk
    sc, ss = si * ck, si * sk

    quat = np.empty(euler.shape[:-1] + (4,), dtype=np.float64)
    quat[..., 0] = cj * cc + sj * ss
    quat[..., 3] = cj * sc - sj * cs
    quat[..., 2] = -(cj * ss + sj * cc)
    quat[..., 1] = cj * cs - sj * sc
    return quat

def mat2bryant(mat):
    """ Convert Rotation Matrix to Euler Angles.  See rotation.py for notes """
    mat = np.asarray(mat, dtype=np.float64)
    assert mat.shape[-2:] == (3, 3), "Invalid shape matrix {}".format(mat)

    cy = np.sqrt(mat[..., 2, 2] * mat[..., 2, 2] + mat[..., 1, 2] * mat[..., 1, 2])
    condition = cy > _EPS4
    euler = np.empty(mat.shape[:-1], dtype=np.float64)
    euler[..., 2] = np.where(condition,
                             -np.arctan2(mat[..., 0, 1], mat[..., 0, 0]),
                             -np.arctan2(-mat[..., 1, 0], mat[..., 1, 1]))
    euler[..., 1] = np.where(condition,
                             -np.arctan2(-mat[..., 0, 2], cy),
                             -np.arctan2(-mat[..., 0, 2], cy))
    euler[..., 0] = np.where(condition,
                             -np.arctan2(mat[..., 1, 2], mat[..., 2, 2]),
                             0.0)
    return euler
def quat2mat(quat):
    """ Convert Quaternion to Euler Angles.  See rotation.py for notes """
    quat = np.asarray(quat, dtype=np.float64)
    assert quat.shape[-1] == 4, "Invalid shape quat {}".format(quat)

    w, x, y, z = quat[..., 0], quat[..., 1], quat[..., 2], quat[..., 3]
    Nq = np.sum(quat * quat, axis=-1)
    s = 2.0 / Nq
    X, Y, Z = x * s, y * s, z * s
    wX, wY, wZ = w * X, w * Y, w * Z
    xX, xY, xZ = x * X, x * Y, x * Z
    yY, yZ, zZ = y * Y, y * Z, z * Z

    mat = np.empty(quat.shape[:-1] + (3, 3), dtype=np.float64)
    mat[..., 0, 0] = 1.0 - (yY + zZ)
    mat[..., 0, 1] = xY - wZ
    mat[..., 0, 2] = xZ + wY
    mat[..., 1, 0] = xY + wZ
    mat[..., 1, 1] = 1.0 - (xX + zZ)
    mat[..., 1, 2] = yZ - wX
    mat[..., 2, 0] = xZ - wY
    mat[..., 2, 1] = yZ + wX
    mat[..., 2, 2] = 1.0 - (xX + yY)
    return np.where((Nq > _FLOAT_EPS)[..., np.newaxis, np.newaxis], mat, np.eye(3))

# forworad kinematics

#### Module overview — forward kinematics
This is the same forward kinematics as the first lab.

Complete the following code for the forward kinematics (FR). This function should compute the end effector’s pose $(x, y, \theta)$ from the joint angles. The end effector is rigidly attached to the third link; model it as a fork-shaped tool whose bottom edge is anchored at the tip of link 3 (see the example figure in the slides). Your implementation return should include the coordinates of the four corner points of the end effector’s rectangular section.

**func**: forward_kinematics:\
**input**: `q`: three joint angles \
**output**: `o`: the pose $(x, y, \theta)$ of first joint, `p`: second joint, `q`: third joint, `e`: end effector; \
`e_left1, e_left2, e_right1, e_right2`: four corners of end effector $(x, y)$ \
You can use the object `robot` and store your useful result during calculation, since the function `__setattr__` has been defined.

In [ ]:
def forward_kinematics(q):

    # [To Be Implemented]

    #this part need to be updated on case by case basis
    # o = robot.body[1].H_global[:3,3]
    # p = robot.body[2].H_global[:3,3]
    # q = robot.body[3].H_global[:3,3]
    # end_eff_pos_local = robot.params.end_eff_pos_local
    # e = robot.body[3].H_global @ np.append(end_eff_pos_local, 1)
    # e = e[:3]

    # e_left1 = E_left1[0:2];
    # e_right1 = E_right1[0:2];
    # e_left2 = E_left2[0:2];
    # e_right2 = E_right2[0:2];

    # theta = np.arccos(H03[0,0])


    # Store in a SimpleNamespace
    sol = SimpleNamespace(
        o=np.array(o),
        p=np.array(p),
        q=np.array(q),
        e=np.array(e),
        e_left1=np.array(e_left1),
        e_right1 = np.array(e_right1),
        e_left2=np.array(e_left2),
        e_right2 = np.array(e_right2),
        theta = theta,
        )

    return sol

# Inverse Kinematics

Implementation of inverse kinematics

In [ ]:
def inverse_kinematics(q,parms):

    x_ref = parms[0]
    y_ref = parms[1]
    theta_ref = parms[2]

    sol = forward_kinematics(q)
    e = sol.e
    theta = sol.theta
    x = e[0]
    y = e[1]
    return x-x_ref,y-y_ref,theta-theta_ref

# Interpretation

This function builds a quintic (5th-order) time polynomial that moves each element of p0 to the corresponding element of pf over the time window [t0, tf] with zero velocity and zero acceleration at both ends.

---

In [ ]:
import numpy as np

def quintic_interp(t,t0,tf,p0,pf):

    p = np.zeros(len(p0))
    pdot = np.zeros(len(p0))
    pddot = np.zeros(len(p0))
    for i in range(0,len(p)):
        if (t<=t0):
            p[i] = p0[i]
        elif (t>=tf):
            p[i] = pf[i]
        else:
            q0 = p0[i];
            qf = pf[i];
            a0 = q0*(-10*t0**2*tf**3 + 5*t0*tf**4 - tf**5)/(t0**5 - 5*t0**4*tf + 10*t0**3*tf**2 - 10*t0**2*tf**3 + 5*t0*tf**4 - tf**5) + qf*(t0**5 - 5*t0**4*tf + 10*t0**3*tf**2)/(t0**5 - 5*t0**4*tf + 10*t0**3*tf**2 - 10*t0**2*tf**3 + 5*t0*tf**4 - tf**5)
            a1 = 30*q0*t0**2*tf**2/(t0**5 - 5*t0**4*tf + 10*t0**3*tf**2 - 10*t0**2*tf**3 + 5*t0*tf**4 - tf**5) - 30*qf*t0**2*tf**2/(t0**5 - 5*t0**4*tf + 10*t0**3*tf**2 - 10*t0**2*tf**3 + 5*t0*tf**4 - tf**5)
            a2 = q0*(-30*t0**2*tf - 30*t0*tf**2)/(t0**5 - 5*t0**4*tf + 10*t0**3*tf**2 - 10*t0**2*tf**3 + 5*t0*tf**4 - tf**5) + qf*(30*t0**2*tf + 30*t0*tf**2)/(t0**5 - 5*t0**4*tf + 10*t0**3*tf**2 - 10*t0**2*tf**3 + 5*t0*tf**4 - tf**5)
            a3 = q0*(10*t0**2 + 40*t0*tf + 10*tf**2)/(t0**5 - 5*t0**4*tf + 10*t0**3*tf**2 - 10*t0**2*tf**3 + 5*t0*tf**4 - tf**5) + qf*(-10*t0**2 - 40*t0*tf - 10*tf**2)/(t0**5 - 5*t0**4*tf + 10*t0**3*tf**2 - 10*t0**2*tf**3 + 5*t0*tf**4 - tf**5)
            a4 = q0*(-15*t0 - 15*tf)/(t0**5 - 5*t0**4*tf + 10*t0**3*tf**2 - 10*t0**2*tf**3 + 5*t0*tf**4 - tf**5) + qf*(15*t0 + 15*tf)/(t0**5 - 5*t0**4*tf + 10*t0**3*tf**2 - 10*t0**2*tf**3 + 5*t0*tf**4 - tf**5)
            a5 = 6*q0/(t0**5 - 5*t0**4*tf + 10*t0**3*tf**2 - 10*t0**2*tf**3 + 5*t0*tf**4 - tf**5) - 6*qf/(t0**5 - 5*t0**4*tf + 10*t0**3*tf**2 - 10*t0**2*tf**3 + 5*t0*tf**4 - tf**5)

            p[i] = a0+a1*t+a2*t**2+a3*t**3+a4*t**4+a5*t**5;
            pdot[i] = a1+2*a2*t+3*a3*t**2+4*a4*t**3+5*a5*t**4;
            pddot[i] = 2*a2+6*a3*t+12*a4*t**2+20*a5*t**3;

    return p,pdot,pddot

## Simulation

In [ ]:

XML = r"""<mujoco>
  <option gravity = "0 0 0"/>
   <worldbody>
      <light diffuse="0 0 0" pos="0 0 10" dir="0 0 -1"/>
      <geom type="plane" size="2.5 2.5 0.01" rgba="1 1 1 1" pos = "0 0 -0.1"/>
      <body name = "body1" pos="0 0 0">
        <inertial mass="1" pos="0.5 0 0" fullinertia="1 1 0.1 0 0 0"/>
         <joint name="joint1" type="hinge" axis = "0 0 1" pos = "0 0 0"/>
         <geom type="box" size="0.5 0.05 0.05" rgba="0.9 0 0 1" pos="0.5 0 0"/>
         <body name = "body2" pos="1 0 0.1">
           <inertial mass="1" pos="0.5 0 0" fullinertia="1 1 0.1 0 0 0"/>
           <joint name="joint2" type="hinge" axis = "0 0 1" pos = "0 0 0"/>
           <geom type="box" size="0.5 0.05 0.05" rgba="0 0 0.9 1" pos="0.5 0 0"/>
           <body name = "body3" pos="1 0 0.1">
             <inertial mass="0.2" pos="0.125 0 0" fullinertia="0.1 0.1 0.02 0 0 0"/>
             <joint name="joint3" type="hinge" axis = "0 0 1" pos = "0 0 0"/>
             <geom type="box" size="0.125 0.05 0.05" rgba="0 0.9 0 1" pos="0.125 0 0"/>
             <site name="tip" pos="0.25 0 0" size="0.01" rgba="1 0 0 1"/>
           </body>
         </body>
      </body>
      <body name="reference" pos="0 0 0" mocap="true">
        <geom type="box" size=".05 .05 .05" rgba=".9 .9 0 1"/>
      </body>
      <body name = "via0" pos="1.0 1.5 0.2"  euler="0 0 90">
        <geom type="box" size="0.125 0.05 0.05" rgba=".1 .1 .1 1" pos="-0.125 0 0"/>
      </body>
      <body name = "via1" pos="1.0 0.75 0.2"  euler="0 0 45">
        <geom type="box" size="0.125 0.05 0.05" rgba=".1 .1 .1 1" pos="-0.125 0 0"/>
      </body>
      <body name = "via2" pos="-0.5 0.5 0.2"  euler="0 0 90">
        <geom type="box" size="0.125 0.05 0.05" rgba=".1 .1 .1 1" pos="-0.125 0 0"/>
      </body>
      <body name = "via3" pos="-1 1.5 0.2"  euler="0 0 180">
        <geom type="box" size="0.125 0.05 0.05" rgba=".1 .1 .1 1" pos="-0.125 0 0"/>
      </body>
   </worldbody>
   <sensor>
     <framepos objtype="site" objname="tip"/>
   </sensor>
   <actuator>
     <position name="joint1" joint="joint1" kp="10" kv="10"/>
     <position name="joint2" joint="joint2" kp="10" kv="10"/>
     <position name="joint3" joint="joint3" kp="10" kv="10"/>
   </actuator>
   <keyframe>
     <key name="home" qpos="0 0 0" ctrl="1.57 -1.57 1.57"/>
   </keyframe>
</mujoco>
"""

print_camera_config = False

dt1, dt2, dt3 = 2.0, 2.0, 2.0
t0 = 1.0
t1, t2, t3 = t0 + dt1, t0 + dt1 + dt2, t0 + dt1 + dt2 + dt3
simend = t3 + 1.0

flag_cubic_spline = 1

cam_cfg = dict(
    azimuth = 89.8231032550546,
    elevation = -88.84857804659835,
    distance = 6.025299868401033,
    lookat = np.array([0.0, 0.0, 0.0], dtype=float),
)


In [ ]:
model = mj.MjModel.from_xml_string(XML)
data  = mj.MjData(model)

try:
    key_id   = model.key('home').id
    key_qpos = model.key_qpos[key_id].copy()
    key_ctrl = model.key_ctrl[key_id].copy()
except Exception:
    key_qpos = data.qpos.copy()
    key_ctrl = data.ctrl.copy() if data.ctrl.size > 0 else np.array([])
    print("cannot find keyframe 'home', use current qpos as the initial value.")

mocap_id = model.body('reference').mocapid[0]

z_ref = 0.0
via_names = ['via0', 'via1', 'via2', 'via3']
via_pos = [model.body(n).pos.copy() for n in via_names]
via_quat = [model.body(n).quat.copy() for n in via_names]
X0_ref, X1_ref, X2_ref, X3_ref = [np.array([p[0], p[1], z_ref]) for p in via_pos]
quat0, quat1, quat2, quat3 = via_quat
euler0 = quat2bryant(quat0)
euler1 = quat2bryant(quat1)
euler2 = quat2bryant(quat2)
euler3 = quat2bryant(quat3)

if flag_cubic_spline == 1:
    t_sp = np.array([0.0, t0, t1, t2, t3, simend])
    x_sp = np.array([X0_ref[0], X0_ref[0], X1_ref[0], X2_ref[0], X3_ref[0], X3_ref[0]])
    y_sp = np.array([X0_ref[1], X0_ref[1], X1_ref[1], X2_ref[1], X3_ref[1], X3_ref[1]])
    cs_x = CubicSpline(t_sp, x_sp, bc_type='natural')
    cs_y = CubicSpline(t_sp, y_sp, bc_type='natural')


In [ ]:
time_all = []
X_all    = np.empty((0,3))  # ref: [x_ref, y_ref, theta_ref]
Xref_all = np.empty((0,3))  # act: [x_act, y_act, theta_act]

q = key_qpos.copy()
dt = model.opt.timestep

with mjv.launch_passive(model, data) as viewer:
    viewer.cam.azimuth   = cam_cfg['azimuth']
    viewer.cam.elevation = cam_cfg['elevation']
    viewer.cam.distance  = cam_cfg['distance']
    viewer.cam.lookat[:] = cam_cfg['lookat']

    t = 0.0
    while viewer.is_running() and t < simend:
        if flag_cubic_spline == 1:
            pos = np.array([cs_x(t), cs_y(t), 0.0], dtype=float)
        else:
            if t < t1:
                pos,_,_ = quintic_interp(t, t0, t1, X0_ref, X1_ref)
            elif t < t2:
                pos,_,_ = quintic_interp(t, t1, t2, X1_ref, X2_ref)
            else:
                pos,_,_ = quintic_interp(t, t2, t3, X2_ref, X3_ref)
            pos = np.asarray(pos, dtype=float)

        if t < t1:
            ang,_,_ = quintic_interp(t, t0, t1, euler0, euler1)
        elif t < t2:
            ang,_,_ = quintic_interp(t, t1, t2, euler1, euler2)
        else:
            ang,_,_ = quintic_interp(t, t2, t3, euler2, euler3)
        ang  = np.asarray(ang, dtype=float)
        quat = bryant2quat(ang)

        data.mocap_pos[mocap_id]  = pos
        data.mocap_quat[mocap_id] = quat

        x_ref, y_ref, theta_ref = pos[0], pos[1], ang[2]
        parms = np.array([x_ref, y_ref, theta_ref], dtype=float)
        q = fsolve(inverse_kinematics, q, args=(parms,))
        
        data.qpos[:] = q
        data.time = t
        mj.mj_forward(model, data)

        if t0 <= t <= t3:
            X_site = data.site('tip').xpos.copy()
            site_quat = np.zeros(4)
            mj.mju_mat2Quat(site_quat, data.site('tip').xmat)
            Ang_site = quat2bryant(site_quat)

            time_all.append(t - t0)
            X_all    = np.vstack([X_all,    np.array([x_ref, y_ref, theta_ref])])  # ref
            Xref_all = np.vstack([Xref_all, np.array([X_site[0], X_site[1], Ang_site[2]])])  # act

        if print_camera_config:
            print('cam.azimuth =', viewer.cam.azimuth,
                  '; cam.elevation =', viewer.cam.elevation,
                  '; cam.distance =', viewer.cam.distance)
            print('cam.lookat =', viewer.cam.lookat)

        viewer.sync()
        t += dt

print('Simulation finished. The number of simulated data points', len(time_all))


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

time_all = np.asarray(time_all)
if time_all.size > 0:
    plt.figure(figsize=(7,4))
    plt.plot(time_all, X_all[:,0], label='ref x')
    plt.plot(time_all, Xref_all[:,0], label='act x')
    plt.xlabel('Time [s]')
    plt.ylabel('x')
    plt.grid(True)
    plt.legend()

    plt.figure(figsize=(7,4))
    plt.plot(time_all, X_all[:,1], label='ref y')
    plt.plot(time_all, Xref_all[:,1], label='act y')
    plt.xlabel('Time [s]')
    plt.ylabel('y')
    plt.grid(True)
    plt.legend()

    plt.figure(figsize=(7,4))
    plt.plot(time_all, X_all[:,2], label='ref theta')
    plt.plot(time_all, Xref_all[:,2], label='act theta')
    plt.xlabel('Time [s]')
    plt.ylabel('theta (rad)')
    plt.grid(True)
    plt.legend()
else:
    print('No data to plot!')
